## Embedding strategies:


### Break large text document into chunks.

Full pipeline code example.

In [33]:
# Import torch and faiss for model and vector indexing.
import torch
import faiss

# Import time for performance measurement.
import time

# Import other required libraries.
import numpy as np
import pandas as pd

# Import SentenceTransformer for embedding generation.
from sentence_transformers import SentenceTransformer

# Ignore warnings for cleaner output.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Import pickle for saving and loading objects.
import pickle

# Import sqlite3 for database operations.
import sqlite3

import nltk
from nltk.tokenize import sent_tokenize

# Download punkt if not already
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/himanshusharma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load documents.

In [34]:
# -------------------------------------------------
# 1. Load your dataset
# -------------------------------------------------
# Example: DataFrame with 'doc_id' and 'text'L
data = pd.DataFrame({
    "doc_id": [1, 2, 3],
    "text": [
        "It is my pleasure to welcome you to this second regular session of the UNICEF Executive Board for 2025. I begin by extending my sincere gratitude to Ambassador Hikmat, President of the Executive Board, and to the entire Bureau for your steady leadership and support.We meet at a time of profound challenge for the world’s children. Cuts to global assistance are putting young lives and futures at risk just when they need us most. Many governments have already reduced their development and humanitarian budgets, and UNICEF expects at least a 20 per cent drop in income over the coming four years.Children are growing up amidst multiple, overlapping threats — conflict, displacement, climate shocks, poverty and discrimination — with too few safety nets to protect them. Essential services that keep them alive, healthy, and safe are being cut back or shut down.According to a recent study in The Lancet, cuts in development and humanitarian aid announced in 2025 could result in an additional 14 million deaths globally by 2030 — including 4.5 million children under the age of five. These are not distant risks. They are the immediate, devastating consequences when health systems, nutrition programmes, education services and clean water are taken away from the youngest members of society.When children lose access to vaccines, diseases spread across borders. When they go without nutrition, education or protection, the desperation that follows fuels instability and displacement that ripple outward. The more fragile children’s lives become, the more fragile the world becomesYet we also know something else: when we invest in children, the returns are extraordinary. Decades of evidence prove that smart investments in child health, education, protection and resilience do not just save lives — they build stronger, safer and more prosperous societies.That is why the discussions we will have this week on our new Strategic Plan and the Integrated Budget could not be more important. These are the tools that will shape how UNICEF continues to deliver for children in this most difficult environment.",
        "Excellencies … UNICEF is already one of the most cost-effective organizations in the global development and humanitarian system. But in the face of the steepest drop in projected external funding in memory, Through our Future Focus Initiative, we are strategically repositioning UNICEF to continue to deliver for every child, especially the most vulnerable, in a rapidly changing world … and to implement the ambitious targets set out in our new Strategic Plan in a resource constrained environment. We are taking bold and necessary steps to become leaner, We are also transforming our approach to delivering technical assistance for our programmes to be even faster, more context-specific and closer to children. This includes establishing four Centres of Excellence to provide high-quality, tailored technical assistance globally, driving more substantial and sustainable impacts at scale. more agile, and closer to the children we serve. we are not standing still.Here is the second document. It also has multiple paragraphs. Semantic search should work well.",
        "तीन यादगार दिनों तक मुझे देखभाल करने वाले परिवारों, बच्चों और युवाओं, समुदाय और सरकारी नेताओं, और हमारे समर्पित यूनिसेफ बिहार के साथियों के साथ चलने का अवसर मिला। मैंने देखा कि कैसे एक साथ मिलकर हम बच्चों के जीवन में स्थायी बदलाव ला सकते हैं। और मैंने महसूस किया कि जब हम बच्चों को सशक्त बनाते हैं, तो वे अपने परिवारों और समुदायों को भी सशक्त बनाते हैं। बिहार में, मैंने देखा कि कैसे शिक्षा, स्वास्थ्य देखभाल, पोषण, और संरक्षण के क्षेत्र में हमारे प्रयास बच्चों के जीवन को बेहतर बना रहे हैं। मैंने उन बच्चों से मुलाकात की जो अब स्कूल जा रहे हैं, जो स्वस्थ और खुश हैं, और जो अपने सपनों को पूरा करने के लिए प्रेरित हैं। मैंने उन परिवारों से भी मुलाकात की जो अब अपने बच्चों के लिए बेहतर भविष्य की उम्मीद कर रहे हैं। मैंने सरकारी अधिकारियों से भी मुलाकात की जो बच्चों के अधिकारों को सुनिश्चित करने के लिए प्रतिबद्ध हैं। मैं यूनिसेफ बिहार के समर्पित कर्मचारियों से भी मिली जो हर दिन बच्चों के लिए काम कर रहे हैं। मैं उनके समर्पण और जुनून से प्रेरित हुई। बिहार में मेरा अनुभव यादगार और प्रेरणादायक रहा। मैंने देखा कि कैसे एक साथ मिलकर हम बच्चों के जीवन में स्थायी बदलाव ला सकते हैं। और मैंने महसूस किया कि जब हम बच्चों को सशक्त बनाते हैं, तो वे अपने परिवारों और समुदायों को भी सशक्त बनाते हैं। मैं यूनिसेफ के साथ काम करने के लिए आभारी हूँ और मैं बिहार में बच्चों के लिए बेहतर भविष्य बनाने के लिए प्रतिबद्ध हूँ।"
    ]
})

In [35]:
data.head()

,doc_id,text
0,1,It is my pleasure to welcome you to this secon...
1,2,Excellencies … UNICEF is already one of the mo...
2,3,तीन यादगार दिनों तक मुझे देखभाल करने वाले परिव...


In [36]:
# -------------------------------------------------
# 2. Load Sentence Transformer model
# -------------------------------------------------
# Load a pre-trained SentenceTransformer model
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)

In [37]:
embedding_dim = model.get_sentence_embedding_dimension()
embedding_dim

384

In [38]:
# -------------------------------------------------
# 3. Function: Split into chunks
# -------------------------------------------------
def chunk_document(text, doc_id, chunk_size=1):
    sentences = sent_tokenize(text)
    chunks = [" ".join(sentences[i:i+chunk_size]) for i in range(0, len(sentences), chunk_size)]
    return [{"doc_id": doc_id, "chunk_id": idx, "chunk": chunk} for idx, chunk in enumerate(chunks)]

In [41]:
# Apply chunking
chunked_docs = []
for _, row in data.iterrows():
    chunked_docs.extend(chunk_document(row["text"], row["doc_id"], chunk_size=1))

chunk_df = pd.DataFrame(chunked_docs)
print("Chunked DataFrame:\n", chunk_df)

Chunked DataFrame:
     doc_id  chunk_id                                              chunk
0        1         0  It is my pleasure to welcome you to this secon...
1        1         1  I begin by extending my sincere gratitude to A...
2        1         2  Cuts to global assistance are putting young li...
3        1         3  Many governments have already reduced their de...
4        1         4  Essential services that keep them alive, healt...
5        1         5                       These are not distant risks.
6        1         6  They are the immediate, devastating consequenc...
7        1         7  When they go without nutrition, education or p...
8        1         8  The more fragile children’s lives become, the ...
9        1         9  Decades of evidence prove that smart investmen...
10       1        10  These are the tools that will shape how UNICEF...
11       2         0  Excellencies … UNICEF is already one of the mo...
12       2         1  But in the face of the

In [42]:
chunk_df

,doc_id,chunk_id,chunk
0,1,0,It is my pleasure to welcome you to this secon...
1,1,1,I begin by extending my sincere gratitude to A...
2,1,2,Cuts to global assistance are putting young li...
3,1,3,Many governments have already reduced their de...
4,1,4,"Essential services that keep them alive, healt..."
5,1,5,These are not distant risks.
6,1,6,"They are the immediate, devastating consequenc..."
7,1,7,"When they go without nutrition, education or p..."
8,1,8,"The more fragile children’s lives become, the ..."
9,1,9,Decades of evidence prove that smart investmen...


### Embedding using batches.

In [43]:
# Batch encoding function
def encode_in_batches(texts, batch_size=128):
    for i in range(0, len(texts), batch_size):
        yield texts[i:i+batch_size]

In [44]:
chunk_df['chunk'].tolist()

['It is my pleasure to welcome you to this second regular session of the UNICEF Executive Board for 2025.',
 'I begin by extending my sincere gratitude to Ambassador Hikmat, President of the Executive Board, and to the entire Bureau for your steady leadership and support.We meet at a time of profound challenge for the world’s children.',
 'Cuts to global assistance are putting young lives and futures at risk just when they need us most.',
 'Many governments have already reduced their development and humanitarian budgets, and UNICEF expects at least a 20 per cent drop in income over the coming four years.Children are growing up amidst multiple, overlapping threats — conflict, displacement, climate shocks, poverty and discrimination — with too few safety nets to protect them.',
 'Essential services that keep them alive, healthy, and safe are being cut back or shut down.According to a recent study in The Lancet, cuts in development and humanitarian aid announced in 2025 could result in an

In [45]:
# Setup FAISS index
embedding_dim = model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(embedding_dim)

In [46]:
# Benchmark: Encoding 
start_time = time.time()
batch_size = 256
for batch in encode_in_batches(chunk_df['chunk'].tolist(), batch_size=batch_size):
    embeddings = model.encode(batch, batch_size=batch_size, convert_to_numpy=True, show_progress_bar=True)
    embeddings = np.array(embeddings, dtype="float32")
    index.add(embeddings)
elapsed = time.time() - start_time
print(f"\nEncoding 2000 sentences took: {elapsed:.2f} seconds")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Encoding 2000 sentences took: 0.82 seconds


In [47]:
print("FAISS index contains", index.ntotal, "vectors")

FAISS index contains 20 vectors


In [48]:
# -------------------------------------------------
# 6. Search Function
# -------------------------------------------------
def semantic_search(query, top_k=3):
    query_emb = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_emb, top_k)

    results = []
    for idx, dist in zip(indices[0], distances[0]):
        row = chunk_df.iloc[idx]
        results.append({
            "doc_id": row["doc_id"],
            "chunk_id": row["chunk_id"],
            "chunk": row["chunk"],
            "score": float(dist)
        })
    return results

In [56]:
# -------------------------------------------------
# 7. Run Example Search
# -------------------------------------------------
query = "nutrition programmes, education services and clean water"
results = semantic_search(query, top_k=5)

In [57]:
print("\nSearch Results:")
for r in results:
    print(f"Doc {r['doc_id']} | Chunk {r['chunk_id']} | Score: {r['score']:.4f}")
    print("Text:", r["chunk"])
    print("---")


Search Results:
Doc 1 | Chunk 9 | Score: 24.1218
Text: Decades of evidence prove that smart investments in child health, education, protection and resilience do not just save lives — they build stronger, safer and more prosperous societies.That is why the discussions we will have this week on our new Strategic Plan and the Integrated Budget could not be more important.
---
Doc 3 | Chunk 0 | Score: 24.2050
Text: तीन यादगार दिनों तक मुझे देखभाल करने वाले परिवारों, बच्चों और युवाओं, समुदाय और सरकारी नेताओं, और हमारे समर्पित यूनिसेफ बिहार के साथियों के साथ चलने का अवसर मिला। मैंने देखा कि कैसे एक साथ मिलकर हम बच्चों के जीवन में स्थायी बदलाव ला सकते हैं। और मैंने महसूस किया कि जब हम बच्चों को सशक्त बनाते हैं, तो वे अपने परिवारों और समुदायों को भी सशक्त बनाते हैं। बिहार में, मैंने देखा कि कैसे शिक्षा, स्वास्थ्य देखभाल, पोषण, और संरक्षण के क्षेत्र में हमारे प्रयास बच्चों के जीवन को बेहतर बना रहे हैं। मैंने उन बच्चों से मुलाकात की जो अब स्कूल जा रहे हैं, जो स्वस्थ और खुश हैं, और जो अपने सपनों

*******